In [3]:
import pandas as pd  #データの整理
import numpy as np
from scipy.optimize import curve_fit
import matplotlib.pyplot as plt
from matplotlib.backends.backend_pdf import PdfPages
from IPython.display import clear_output

df_gaia = pd.read_csv("simultaneous_event_candidates_table_MU+Tomo-e9_gaia.csv")
df_ucac = pd.read_csv("simultaneous_event_candidates_table_MU+Tomo-e9_ucac.csv")
df_gaia = df_gaia.rename(columns={"star_number": "star_number_gaia"})
df_gaia = df_gaia.rename(columns={"zero_mag": "zero_mag_gaia"})
df_ucac = df_ucac.rename(columns={"star_number": "star_number_ucac"})
df_ucac = df_ucac.rename(columns={"zero_mag": "zero_mag_ucac"})

columns = df_gaia.columns
df_gaia = df_gaia.values
df_gaia = np.delete(df_gaia,np.arange(141),1)
columns = np.delete(columns,np.arange(141))
df_gaia = pd.DataFrame(data=df_gaia,columns=columns)
df_concat = pd.concat([df_ucac, df_gaia], axis=1)
columns_concat = df_concat.columns
array_concat = df_concat.values

def gauss(x, A, mu, sigma):
    return A / np.sqrt(2.0*np.pi) / sigma * np.exp(-((x-mu)/sigma)**2/2)

for i in  range(len(columns)):  # IDの前にある不要カラムを削除
    if columns_concat[i] == "ID":
        delete_column = i
array_concat = np.delete(array_concat,np.arange(delete_column),1)
columns_concat = np.delete(columns_concat,np.arange(delete_column))

### 14等級以上のものは一回リムーブ、IDをリチェックリストにして出す
recheck_index = np.where(array_concat[:,-6]>14.)
recheck_ID = array_concat[recheck_index,0]
recheck_ID = recheck_ID.transpose()
recheck_ID = recheck_ID.reshape(len(recheck_ID))
array_concat = np.delete(array_concat,recheck_index,0)
df_main = pd.DataFrame(data=array_concat,columns=columns_concat)
with open("recheck_list.txt", mode='w') as f:
    f.write("G_Magnitudeが14以上のID\n")
    for i in recheck_ID:
        state = str(i) + "\n"
        f.write(state)

###  theta < 2.5deg], distance < 0.25[deg], full overlapのやつ
for i in range(len(columns_concat)):
    if columns_concat[i] == "distance":
        index_distance = i
    if columns_concat[i] == "theta":
        index_theta = i
    if columns_concat[i] == "percentage":
        index_percentage = i

array_full_overlapping = np.delete(array_concat,np.where(array_concat[:,index_distance]>0.25),0)
array_full_overlapping = np.delete(array_full_overlapping,np.where(array_full_overlapping[:,index_theta]>2.5),0)
array_sim = np.delete(array_concat,np.where(array_concat[:,index_distance]>0.25),0)
array_sim = np.delete(array_sim,np.where(array_sim[:,index_theta]>2.5),0)
array_full_overlapping = np.delete(array_full_overlapping,np.where(array_full_overlapping[:,index_percentage]<100.),0)
df_full_overlapping = pd.DataFrame(data=array_full_overlapping,columns=columns_concat)
df_sim = pd.DataFrame(data=array_sim,columns=columns_concat)

# G-band2V-band
# V =  0.953*G +0.793
df_main["v_band_abs_mag_meteor_coverted"] = 0.953*df_main["g_band_abs_mag_meteor"] +0.793
df_sim["v_band_abs_mag_meteor_coverted"] = 0.953*df_sim["g_band_abs_mag_meteor"] +0.793
df_full_overlapping["v_band_abs_mag_meteor_coverted"] =0.953*df_full_overlapping["g_band_abs_mag_meteor"] +0.793

df_main.to_csv("simultaneous_event_candidates_table_MU+Tomo-e10.csv")
df_sim.to_csv("MU+Tomo-e_main.csv") #同時流星
df_full_overlapping.to_csv("simultaneous_event_candidates_table_MU+Tomo-e10_full.csv") #同時流星&overlapping
#class_width = 1. + np.log2(len(array_concat))
#print class_width
print "Complete!"

Complete!


In [8]:
import pandas as pd #サイズ焼き直し
import numpy as np

df_main = pd.read_csv("simultaneous_event_candidates_table_MU+Tomo-e10.csv")
df_mu = pd.read_csv("../../iccd_2018/MU_data_2009-2015.csv")
df_quadra = pd.read_csv("../../d_criterion/quadra(0.2).csv")
df_perseids = pd.read_csv("../../d_criterion/perseids(0.2).csv")
df_orionid = pd.read_csv("../../d_criterion/orionid(0.2).csv")
df_draconids = pd.read_csv("../../d_criterion/draconids(0.2).csv")
df_leonid = pd.read_csv("../../d_criterion/leonid(0.2).csv")
df_geminid = pd.read_csv("../../d_criterion/geminid(0.2).csv")
df_perseids_zeta = pd.read_csv("../../d_criterion/perseids_zeta(0.2).csv")
df_aquariids_delta = pd.read_csv("../../d_criterion/aquariids_delta(0.2).csv")
df_aquariids_ita = pd.read_csv("../../d_criterion/aquariids_ita(0.2).csv")
df_monoceros = pd.read_csv("../../d_criterion/monoceros(0.2).csv")

ID_mu = df_mu["ID"].values
ID_quadra = df_quadra["ID"].values
ID_perseids = df_perseids["ID"].values
ID_orionid = df_orionid["ID"].values
ID_draconids = df_draconids["ID"].values
ID_leonid = df_leonid["ID"].values
ID_geminid = df_geminid["ID"].values
ID_perseids_zeta = df_perseids_zeta["ID"].values
ID_aquariids_delta = df_aquariids_delta["ID"].values
ID_aquariids_ita = df_aquariids_ita["ID"].values
ID_monoceros = df_monoceros["ID"].values
ID_meteorstream = np.hstack((ID_quadra,ID_perseids,ID_orionid,ID_draconids,ID_leonid,\
                             ID_geminid,ID_perseids_zeta,ID_aquariids_delta,ID_aquariids_ita,\
                             ID_monoceros))
v_main = df_main["velocity"]
v_mu = df_mu["Geocentric_vel_(km_per_s)"]
el_main = 90. - df_main["Zenith_angle_correction"]
el_mu = 90. - df_mu["Ze_of_start_point_(deg)"]

# Y = -0.118*X + 4.944
#RCS = -4.58*MAG+16.92 ⇨ MAG = RCS/(-4.58) + 16.92
#df_main["mag_calculated"] = -0.118*df_main["RCS_dBsm_at_max_SNR"] + 4.944
#df_mu["mag_calculated"] = -0.118*df_mu["RCS"] + 4.944
#df_geminid["mag_calculated"] = -0.118*df_geminid["RCS"] + 4.944
df_main["mag_calculated"] = df_main["RCS_dBsm_at_max_SNR"]/(-4.58) + 16.92/4.58
df_mu["mag_calculated"] = df_mu["RCS"]/(-4.58) + 16.92/4.58

log_mass_grams_calculated_main = 6.31 - 0.4*df_main["mag_calculated"] - 3.92*np.log10(v_main) -\
                                    0.41*np.log10(np.sin(np.deg2rad(el_main)))
log_mass_grams_calculated_mu =6.31 - 0.4*df_mu["mag_calculated"] - 3.92*np.log10(v_mu) -\
                                    0.41*np.log10(np.sin(np.deg2rad(el_mu)))

df_main["log_mass_grams"] = log_mass_grams_calculated_main
df_mu["log_mass_grams"] = log_mass_grams_calculated_mu

df_main["mass_grams_calculated"] = np.power(10,log_mass_grams_calculated_main)
df_mu["mass_grams_calculated"] = np.power(10,log_mass_grams_calculated_mu)

## assuming ρ = 2.0g/cm^2
## サイズは直径[m]
df_main["size_calculated"] = np.cbrt(6.*df_main["mass_grams_calculated"]/(2.*np.pi))/100.
df_mu["size_calculated"] = np.cbrt(6.*df_mu["mass_grams_calculated"]/(2.*np.pi))/100.
df_main["size_log_calculated"] = np.log10(df_main["size_calculated"])
df_mu["size_log_calculated"] = np.log10(df_mu["size_calculated"])


# MUのデータからmeteorstreamを除いたテーブル
delete_index = []
for i in range(len(ID_meteorstream)):
    for j in range(len(ID_mu)):
        if ID_meteorstream[i] == ID_mu[j]:
            delete_index = np.hstack((delete_index,j))

df_mu_dropped = df_mu.drop(delete_index,axis=0)

table_main = df_main.values
table_mu = df_mu.values
table_mu_dropped = df_mu_dropped.values
columns_main = df_main.columns
columns_mu = df_mu.columns
columns_mu_dropped = df_mu_dropped.columns

for i in  range(len(columns_main)):  # IDの前にある不要カラムを削除
    if columns_main[i] == "ID":
        delete_column_main = i
for j in  range(len(columns_mu)):
    if columns_mu[j] == "ID":
        delete_column_mu = j
for k in  range(len(columns_mu_dropped)):
    if columns_mu_dropped[k] == "ID":
        delete_column_mu_dropped = k        
        
table_main = np.delete(table_main,np.arange(delete_column_main),1)
columns_main = np.delete(columns_main,np.arange(delete_column_main))
table_mu = np.delete(table_mu,np.arange(delete_column_mu),1)
columns_mu = np.delete(columns_mu,np.arange(delete_column_mu))
table_mu_dropped = np.delete(table_mu_dropped,np.arange(delete_column_mu_dropped),1)
columns_mu_dropped = np.delete(columns_mu_dropped,np.arange(delete_column_mu_dropped))

df_main = pd.DataFrame(data=table_main, columns=columns_main)
df_mu = pd.DataFrame(data=table_mu, columns=columns_mu)
df_mu_dropped = pd.DataFrame(data=table_mu_dropped, columns=columns_mu_dropped)

df_main.to_csv("simultaneous_event_candidates_table_MU+Tomo-e10.csv")
df_mu.to_csv("../../iccd_2018/MU_data_2009-2015.csv")
df_mu_dropped.to_csv("../../iccd_2018/MU_data_2009-2015_dropped.csv")

print "Complete!"
# このプログラムを実行してから ../d_criterion のd_criterion.ipynbをやって Geminidの軌道を出す

Complete!
